# ANALYING DATA SET OF IPL AND CREATING FANTASY IPL TEAM

In [120]:
import numpy as np 
import pandas as pd

In [121]:
df1=pd.read_csv("IPL Matches 2008-2020.csv")
df1.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [122]:
df2=pd.read_csv("IPL Ball-by-Ball 2008-2020.csv")
df2.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [123]:
df1.isnull().sum()
df2.isnull().sum()

id                       0
inning                   0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
batsman_runs             0
extra_runs               0
total_runs               0
non_boundary             0
is_wicket                0
dismissal_kind      183973
player_dismissed    183973
fielder             186684
extras_type         183235
batting_team             0
bowling_team           191
dtype: int64

# Fantasy IPL Dream Team

In [124]:
data = pd.merge(df2, df1[['id','date']], on='id',how='left')

data['year'] = pd.to_datetime(data['date']).dt.year
data.drop('date',axis=1,inplace=True)

season_2020 = data[data['year']==2020]
season_2013 = data[data['year']==2013]
tournament = df2.copy()

In [125]:
def batsman_score(x):
    x["bat_score"]=1*x["runs"]+1*x["fours"]+2*x["sixes"]+4*x["thirty"]+8*x["fifty"]+16*x["century"]
    return x

def get_bat_stats(df):
    
    batting_df = pd.DataFrame(columns=["player","runs","fours","sixes","thirty","fifty","century"])

    for player,details in df.groupby("batsman"):
        total_runs = sum(details["batsman_runs"])

        fours=0
        try:
            fours=details["batsman_runs"].value_counts()[4]
        except:
            fours=0      

        sixes=0
        try:
            sixes=details["batsman_runs"].value_counts()[6]
        except:
            sixes=0 

        thirty=0
        fifty=0
        century=0

        for match,match_details in details.groupby("id"):

            match_score = sum(match_details["batsman_runs"])

            if match_score>=100:
                century+=1
            elif match_score>=50:
                fifty+=1
            elif match_score>=30:
                thirty+=1

        batting_df = batting_df.append({"player":player,"runs":total_runs,"fours":fours,"sixes":sixes,"thirty":thirty,"fifty":fifty,"century":century},ignore_index=True)
    
    batting_df=batting_df.apply(batsman_score,axis=1)
    return batting_df

In [126]:
def bowler_score(x):
    x["bowl_score"]=25*x["wickets"]+8*x["lbw_or_bowled"]+4*x["three_wickets"]+8*x["four_wickets"]+16*x["five_wickets"]
    return x

def get_bowl_stats(df):
    bowling_df = pd.DataFrame(columns=["player","wickets","lbw_or_bowled","three_wickets","four_wickets","five_wickets"])

    for player,details in df.groupby("bowler"):

        lbw_or_bowled=len(details.query('dismissal_kind == "lbw" or dismissal_kind == "bowled"'))

        total_wickets=0
        three_wickets=0
        four_wickets=0
        fifer=0

        for match,match_details in details.groupby("id"):
            wickets = len(match_details.query('dismissal_kind in ["caught","bowled","lbw","stumped","caught and bowled"]'))
            if wickets>=3:
                three_wickets+=1
            elif wickets>=4:
                four_wickets+=1
            elif wickets>=5:
                fifer+=1
            total_wickets+=wickets

        bowling_df = bowling_df.append({"player":player,"wickets":total_wickets,"lbw_or_bowled":lbw_or_bowled,"three_wickets":three_wickets,"four_wickets":four_wickets,"five_wickets":fifer},ignore_index=True)
        
    bowling_df=bowling_df.apply(bowler_score,axis=1) 
    return bowling_df

In [127]:
def get_wicketkeepers(df):
    return list(df.query('dismissal_kind == "stumped"')['fielder'].unique())

### for whole tournament

In [128]:
batsmen_tournament = get_bat_stats(tournament)
batsmen_tournament.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score
0,A Ashish Reddy,280,16,15,2,0,0,334
1,A Chandila,4,0,0,0,0,0,4
2,A Chopra,53,7,0,0,0,0,60
3,A Choudhary,25,1,1,0,0,0,28
4,A Dananjaya,4,0,0,0,0,0,4


In [129]:
bowlers_tournament = get_bowl_stats(tournament)
bowlers_tournament.head()

,player,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Ashish Reddy,18,9,1,0,0,526
1,A Chandila,11,0,1,0,0,279
2,A Choudhary,5,0,0,0,0,125
3,A Dananjaya,0,0,0,0,0,0
4,A Flintoff,2,0,0,0,0,50


In [130]:
merged_data_tournament = pd.merge(batsmen_tournament,bowlers_tournament,on="player",how="outer").fillna(0)
merged_data_tournament.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Ashish Reddy,280.0,16.0,15.0,2.0,0.0,0.0,334.0,18.0,9.0,1.0,0.0,0.0,526.0
1,A Chandila,4.0,0.0,0.0,0.0,0.0,0.0,4.0,11.0,0.0,1.0,0.0,0.0,279.0
2,A Chopra,53.0,7.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A Choudhary,25.0,1.0,1.0,0.0,0.0,0.0,28.0,5.0,0.0,0.0,0.0,0.0,125.0
4,A Dananjaya,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
merged_data_tournament['allrounder_score']=(merged_data_tournament['bat_score']*merged_data_tournament['bowl_score'])/1000
merged_data_tournament.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score,allrounder_score
0,A Ashish Reddy,280.0,16.0,15.0,2.0,0.0,0.0,334.0,18.0,9.0,1.0,0.0,0.0,526.0,175.684
1,A Chandila,4.0,0.0,0.0,0.0,0.0,0.0,4.0,11.0,0.0,1.0,0.0,0.0,279.0,1.116
2,A Chopra,53.0,7.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
3,A Choudhary,25.0,1.0,1.0,0.0,0.0,0.0,28.0,5.0,0.0,0.0,0.0,0.0,125.0,3.500
4,A Dananjaya,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000


In [132]:
wicketkeeper_tournament = get_wicketkeepers(tournament)

In [133]:
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_tournament = {}

playing_XI_tournament['BAT'] = merged_data_tournament.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_tournament['WK'] = merged_data_tournament.query('player in @wicketkeeper_tournament').sort_values(by="bat_score",ascending=False).head(total_wicketkeepers)['player'].to_list()
playing_XI_tournament['AR'] = merged_data_tournament.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_tournament['BWL'] = merged_data_tournament.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [149]:
print('Fantasy dream team for the whole tournament\n')
for player_role,players in playing_XI_tournament.items():
    for player in players:
        print("{:15} {}".format(player,player_role))

Fantasy dream team for the whole tournament

V Kohli         BAT
SK Raina        BAT
DA Warner       BAT
RG Sharma       BAT
AB de Villiers  WK
SR Watson       AR
RA Jadeja       AR
DJ Bravo        AR
SL Malinga      BWL
PP Chawla       BWL
A Mishra        BWL


### FOR SEASON 2013

In [135]:
batsmen_2013 = get_bat_stats(season_2013)
batsmen_2013.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score
0,A Ashish Reddy,125,8,5,1,0,0,147
1,A Chandila,4,0,0,0,0,0,4
2,A Mishra,89,9,0,1,0,0,102
3,A Mithun,2,0,0,0,0,0,2
4,A Mukund,19,1,0,0,0,0,20


In [136]:
bowlers_2013 = get_bowl_stats(season_2013)
bowlers_2013.head()

,player,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Ashish Reddy,3,2,0,0,0,91
1,A Chandila,6,0,0,0,0,150
2,A Mishra,21,7,3,0,0,593
3,A Mithun,1,0,0,0,0,25
4,A Nehra,11,1,0,0,0,283


In [137]:
wicketkeeper_2013 = get_wicketkeepers(season_2013)

In [138]:
merged_data_2013 = pd.merge(batsmen_2013,bowlers_2013,on="player",how="outer").fillna(0)
merged_data_2013.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Ashish Reddy,125.0,8.0,5.0,1.0,0.0,0.0,147.0,3.0,2.0,0.0,0.0,0.0,91.0
1,A Chandila,4.0,0.0,0.0,0.0,0.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,150.0
2,A Mishra,89.0,9.0,0.0,1.0,0.0,0.0,102.0,21.0,7.0,3.0,0.0,0.0,593.0
3,A Mithun,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,25.0
4,A Mukund,19.0,1.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
merged_data_2013['allrounder_score']=(merged_data_2013['bat_score']*merged_data_2013['bowl_score'])/1000
merged_data_2013.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score,allrounder_score
0,A Ashish Reddy,125.0,8.0,5.0,1.0,0.0,0.0,147.0,3.0,2.0,0.0,0.0,0.0,91.0,13.377
1,A Chandila,4.0,0.0,0.0,0.0,0.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,150.0,0.600
2,A Mishra,89.0,9.0,0.0,1.0,0.0,0.0,102.0,21.0,7.0,3.0,0.0,0.0,593.0,60.486
3,A Mithun,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,25.0,0.050
4,A Mukund,19.0,1.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000


In [140]:
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_2013 = {}

playing_XI_2013['BAT'] = merged_data_2013.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_2013['WK'] = ['MS Dhoni']
playing_XI_2013['AR'] = merged_data_2013.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_2013['BWL'] = merged_data_2013.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [141]:
print('Fantasy dream team for season 2013\n')
for player_role,players in playing_XI_2013.items():
    for player in players:
        print("{:15} {}".format(player,player_role))

Fantasy dream team for season 2013

CH Gayle        BAT
MEK Hussey      BAT
V Kohli         BAT
SR Watson       BAT
MS Dhoni        WK
SR Watson       AR
JH Kallis       AR
NLTC Perera     AR
DJ Bravo        BWL
JP Faulkner     BWL
Harbhajan Singh BWL


### FOR SEASON 2020

In [142]:
batsmen_2020 = get_bat_stats(season_2020)
batsmen_2020.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score
0,A Nortje,7,0,0,0,0,0,7
1,AB de Villiers,454,33,23,2,5,0,581
2,AD Russell,117,9,9,0,0,0,144
3,AJ Finch,268,28,8,2,1,0,328
4,AJ Tye,6,0,1,0,0,0,8


In [143]:
bowlers_2020 = get_bowl_stats(season_2020)
bowlers_2020.head()

,player,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Mishra,3,0,0,0,0,75
1,A Nortje,22,9,2,0,0,630
2,A Zampa,2,0,0,0,0,50
3,AD Russell,5,1,0,0,0,133
4,AJ Tye,1,0,0,0,0,25


In [144]:
merged_data_2020 = pd.merge(batsmen_2020,bowlers_2020,on="player",how="outer").fillna(0)
merged_data_2020.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Nortje,7.0,0.0,0.0,0.0,0.0,0.0,7.0,22.0,9.0,2.0,0.0,0.0,630.0
1,AB de Villiers,454.0,33.0,23.0,2.0,5.0,0.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AD Russell,117.0,9.0,9.0,0.0,0.0,0.0,144.0,5.0,1.0,0.0,0.0,0.0,133.0
3,AJ Finch,268.0,28.0,8.0,2.0,1.0,0.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AJ Tye,6.0,0.0,1.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,0.0,25.0


In [145]:
merged_data_2020['allrounder_score']=(merged_data_2020['bat_score']*merged_data_2020['bowl_score'])/1000
merged_data_2020.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score,allrounder_score
0,A Nortje,7.0,0.0,0.0,0.0,0.0,0.0,7.0,22.0,9.0,2.0,0.0,0.0,630.0,4.410
1,AB de Villiers,454.0,33.0,23.0,2.0,5.0,0.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
2,AD Russell,117.0,9.0,9.0,0.0,0.0,0.0,144.0,5.0,1.0,0.0,0.0,0.0,133.0,19.152
3,AJ Finch,268.0,28.0,8.0,2.0,1.0,0.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
4,AJ Tye,6.0,0.0,1.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,0.0,25.0,0.200


In [146]:
wicketkeeper_2020 = get_wicketkeepers(season_2020)

In [147]:
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_2020 = {}

playing_XI_2020['BAT'] = merged_data_2020.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_2020['WK'] = merged_data_2020.query('player in @wicketkeeper_2020').sort_values(by="bat_score",ascending=False).head(total_wicketkeepers)['player'].to_list()
playing_XI_2020['AR'] = merged_data_2020.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_2020['BWL'] = merged_data_2020.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [148]:
print('Fantasy dream team for season 2020\n')
for player_role,players in playing_XI_2020.items():
    for player in players:
        print("{:15} {}".format(player,player_role))

Fantasy dream team for season 2020

KL Rahul        BAT
S Dhawan        BAT
DA Warner       BAT
Ishan Kishan    BAT
Q de Kock       WK
MP Stoinis      AR
R Tewatia       AR
SM Curran       AR
K Rabada        BWL
JJ Bumrah       BWL
TA Boult        BWL
